<a href="https://colab.research.google.com/github/ValentinCord/HandsOnAI_2/blob/main/NLP_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <span> NLP : Entrainnement et sauvegarde du modèle LSTM/GRU </span>
<hr style="border-bottom: solid;background-color:light;color:black;">

<a id="section-1"></a>
# <span>1. Installation des packages</span>
<hr style="border-bottom: solid;background-color:light;color:black;">

In [13]:
!/opt/bin/nvidia-smi
!rm -rf sample_data

Fri Dec 23 20:12:05 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   77C    P0    33W /  70W |   4986MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# <span>2. Imports </span>
<hr style="border-bottom: solid;background-color:light;color:black;">

In [14]:
# basics 
import os
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# tensorflow
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam 
from tensorflow.keras.layers import Conv1D, MaxPooling1D

# plot 
import matplotlib.pyplot as plt 
import seaborn as sns 

# nltk 
import re
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


<a id="section-3"></a>
# <span>3. Choix des paramètres</span>
<hr style="border-bottom: solid;background-color:light;color:black;">

In [28]:
MAX_SEQ_LEN = 5000
DONNEE_AJOUTEES = 200
TRAIN_SIZE = 0.8

batch_size = 64
num_epochs = 3

train_path = '/content/drive/MyDrive/HandOnAI_2_NLP/fake_train.csv'
added_path = '/content/drive/MyDrive/HandOnAI_2_NLP/added_train.csv'
test_path = '/content/drive/MyDrive/HandOnAI_2_NLP/fake_test.csv'

<a id="section-4"></a>
# <span>4. Lecture des données</span>
<hr style="border-bottom: solid;background-color:light;color:black;">

In [17]:
df = pd.read_csv(train_path)
df_added = pd.read_csv(added_path)
df_test = pd.read_csv(test_path)

<a id="section-5"></a>
# <span>5. Preprocessing</span>
<hr style="border-bottom: solid;background-color:light;color:black;">

## <span>5.1 Nettoyage de données</span>
<hr style="border-bottom: solid;background-color:light;color:black;">

In [18]:
df = df.drop(['Unnamed: 0', 'target_name'], axis = 1)
df_added.rename(columns = {'french':'data'}, inplace = True)
df_added = df_added.drop(['Unnamed: 0'], axis = 1)
df_test = df_test.drop(['Unnamed: 0', 'target_name'], axis = 1)

df = df.append(df_added[:DONNEE_AJOUTEES], ignore_index=True)

In [19]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
STOPWORDS = set(stopwords.words('french'))

def clean_text(text):
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text

In [20]:
df['data'] = df['data'].apply(clean_text)
df_test['data'] = df_test['data'].apply(clean_text)

df = df.drop(df.index[1136])
df = df.reset_index()

<a id="section-6"></a>
# <span>6. Tokenisation des données</span>
<hr style="border-bottom: solid;background-color:light;color:black;">

In [21]:
df_train, df_val = train_test_split(df, test_size=0.2, shuffle=True)

X_train = df_train.data.tolist()
X_val = df_val.data.tolist()
X_test = df_test.data.tolist()

y_train = df_train.label.tolist()
y_val = df_val.label.tolist()
y_test = df_test.label.tolist()

train_text_vec = [text for text in X_train]
val_text_vec = [text for text in X_val]
test_text_vec = [text for text in X_test]

# tokenize the sentences
tokenizer = Tokenizer(lower=False)
tokenizer.fit_on_texts(train_text_vec)

train_text_vec = tokenizer.texts_to_sequences(train_text_vec)
val_text_vec = tokenizer.texts_to_sequences(val_text_vec)
test_text_vec = tokenizer.texts_to_sequences(test_text_vec)

# pad the sequences
train_text_vec = pad_sequences(train_text_vec, maxlen=MAX_SEQ_LEN)
val_text_vec = pad_sequences(val_text_vec, maxlen=MAX_SEQ_LEN)
test_text_vec = pad_sequences(test_text_vec, maxlen=MAX_SEQ_LEN)


In [22]:
# One Hot Encode Y values:
encoder = LabelEncoder()

y_train = encoder.fit_transform(df_train['label'].values)
y_train = to_categorical(y_train) 

y_val = encoder.fit_transform(df_val['label'].values)
y_val = to_categorical(y_val) 

y_test = encoder.fit_transform(df_test['label'].values)
y_test = to_categorical(y_test) 


<a id="section-7"></a>
# <span>7. Création du modèle</span>
<hr style="border-bottom: solid;background-color:light;color:black;">

In [31]:
model = Sequential()
model.add(Embedding(input_dim = (len(tokenizer.word_counts) + 1), output_dim = 128, input_length = MAX_SEQ_LEN))
model.add(Conv1D(32,3, padding='same'))
model.add(MaxPooling1D())
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dense(2, activation='softmax'))

<a id="section-8"></a>
# <span>8. Entrainement du modèle</span>
<hr style="border-bottom: solid;background-color:light;color:black;">

In [32]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_text_vec, y_train, validation_data=(val_text_vec, y_val), batch_size=batch_size, epochs=num_epochs)

Epoch 1/3
21/21 [==============================] - 5s 167ms/step - loss: 0.6870 - accuracy: 0.5411 - val_loss: 0.5555 - val_accuracy: 0.7801
Epoch 2/3
21/21 [==============================] - 3s 146ms/step - loss: 0.4061 - accuracy: 0.8619 - val_loss: 0.2682 - val_accuracy: 0.9157
Epoch 3/3
21/21 [==============================] - 3s 144ms/step - loss: 0.1058 - accuracy: 0.9758 - val_loss: 0.2310 - val_accuracy: 0.9217


In [33]:
test_scores = model.evaluate(test_text_vec, y_test, verbose=1)

print("test scores:", test_scores)

16/16 [==============================] - 1s 57ms/step - loss: 0.1703 - accuracy: 0.9300
test scores: [0.17031842470169067, 0.9300411343574524]


<a id="section-9"></a>
# <span>9. Sauvegarde du modèle</span>
<hr style="border-bottom: solid;background-color:light;color:black;">

In [35]:
model.save('/content/drive/MyDrive/HandOnAI_2_NLP/LSTM_model.h5')